In [10]:
# 不通过图的方式, 完成一个图基本算法的撰写
import numpy as np
from collections import deque
from typing import Optional

In [ ]:
def build_adjM(vertexNum:int, edges:list[tuple[int, int]])->np.ndarray:
    # vertex_id=[i for i in range(vertexNum)]
    adjM=np.zeros(shape=(vertexNum, vertexNum))

    # 不用numpy库实现 adjM 创建: adjM=[[0 for _ in range(numCourses)] for _ in range(numCourses)]

    for (u, v) in edges:
        adjM[u, v]=adjM[v, u]=1 # 无向无权图

    return adjM

In [12]:
# BFS: my version, 写得没有下面的好
# def BFS_from_certain_node(adjM, start_id, que): # 从某个结点开始 BFS
    
#     if not que:
#         que=deque()

#     vertexNum=np.shape(adjM)[0]

#     mark=np.zeros(shape=vertexNum)
#     dist=np.zeros(shape=vertexNum)
#     pred=np.full(shape=vertexNum, fill_value=-1)

#     que.append(start_id)
#     mark[start_id]=1
    
#     while que:
#         v=que.popleft()
#         mark[v]=1
        
#         # do something to v here...

#         for u in range(vertexNum):
#             if adjM[v, u]==1 and mark[u]==0:
#                 que.append(u)
#                 pred[u]=v
#                 dist[u]=dist[v]+1
#         mark[v]=2

# def BFS_Global(adjM): # 全局BFS
#     que=deque()

#     vertexNum=np.shape(adjM)[0]

#     mark=np.zeros(shape=vertexNum)
#     dist=np.zeros(shape=vertexNum)
#     pred=np.full(shape=vertexNum, fill_value=-1)

#     for v in range(vertexNum):
#         if mark[v]==0:
#             BFS_from_certain_node(adjM, start_id=v, que=[])

In [13]:
def BFS(adjM: np.ndarray, start_id: Optional[int] = None) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    统一的BFS遍历函数,支持指定起点或全局遍历(处理非连通图)
    
    参数:
        adjM: 邻接矩阵 (nxn), 元素为0或1
        start_id: 起始节点ID(0-based),默认None表示全局遍历(处理所有连通分量)
            Optional 是 Python 标准库 typing 模块提供的「类型提示工具」.
            Optional[T] = Union[T, None]
            翻译过来就是:「参数类型可以是 T(这里是 int), 也可以是 None」.

            start_id: Optional[int] = None
                应该拆成两部分看:
                    Optional[int] 如上.
                    = None: 如果不写 start_id 参数 默认 start_id=None


            补充说明:
                Python >=3.10 中, PEP 604 引入了新语法, 支持直接用 | 表示「类型联合」, 此时可以不用 Optional:
                start_id: int | None = None  (等价于 start_id: Optional[int]=None)
            
    返回:
        mark: 标记数组 (0:未访问, 1:访问中, 2:已访问完毕)
        dist: 距离数组,起点到各节点的距离(不可达为-1)
        pred: 前驱节点数组,各节点的前驱节点ID(无前驱为-1)


    复杂度分析:
        只对一个结点v进行BFS, 复杂度 O(1+deg(v)), 其中 deg(v)是 v 的度
        全图 BFS, 复杂度 O(|V|+|E|), |V|, |E|分别表示图的顶点数,边数
    """
    vertexNum = np.shape(adjM)[0]
    if vertexNum == 0:
        return np.array([]), np.array([]), np.array([])
    
    # 初始化返回数组
    mark = np.zeros(shape=vertexNum, dtype=int)
    dist = np.full(shape=vertexNum, fill_value=-1, dtype=int)  # 不可达标记为-1更合理
    pred = np.full(shape=vertexNum, fill_value=-1, dtype=int)
    
    # 核心BFS辅助函数(内部使用)
    def _bfs_helper(start: int):

        '''
        为什么 _bfs_helper 开头有下划线?
        Python 的命名规范(PEP 8)中定义的「私有成员标记」, 
        核心目的是明确函数 / 变量的访问权限, 避免外部误调用

        这里 _bfs_helper 能够访问并且修改外部 mark, dist, pred数组的原因, 是这些因为这些数组通过列表存储
        而列表是可变对象, 因此传这些数组的参数时, 是类似于C 语言传地址的, 因此可以直接修改

        单下划线 _xxx:表示「内部使用的非公有成员」(约定俗成, 不是强制语法限制)
        双下划线 __xxx:会触发 Python 的「名称修饰(name mangling)」, 真正限制外部访问(比如 __foo 会被改成 _类名__foo)
        '''
        que=deque()
        que.append(start)
        mark[start] = 1
        dist[start] = 0  # 起点距离为0
        
        while que:
            v = que.popleft()
            
            # TODO: 在这里添加对节点v的处理逻辑
            # 示例: print(f"Processing node {v}")
            '''
            主流代码编辑器(VS Code、PyCharm、Sublime 等)内置的「注释关键词高亮规则」
            '''
            
            # 遍历所有邻接节点
            for u in range(vertexNum):
                if adjM[v, u] == 1 and mark[u] == 0:
                    que.append(u)
                    mark[u] = 1 # 结点入队, 标记为访问中, 避免重复入队
                    pred[u] = v
                    dist[u] = dist[v] + 1
            mark[v] = 2  # 标记为已处理完毕
    
    # 执行BFS
    if start_id is not None:
        # 模式1:指定起点的BFS
        if 0 <= start_id < vertexNum:
            _bfs_helper(start_id)
        else:
            raise ValueError(f"start_id必须在[0, {vertexNum-1}]范围内")
    else:
        # 模式2:全局BFS(处理所有连通分量)
        for v in range(vertexNum):
            if mark[v] == 0:
                _bfs_helper(v)

    # 如果进行了全局BFS, 那返回的dist 应该是到各自连通分量上一点的距离
    # print(mark, dist, pred)
    return mark, dist, pred

In [14]:
'''
自己写的不太好的代码
def DFS_from_certain_node(adjM:np.ndarray, start_id): # 从某个结点开始DFS
    vertexNum=np.shape(adjM)[0]

    mark=np.zeros(shape=vertexNum)
    pred=np.full(shape=vertexNum, fill_value=-1)
    d=np.zeros(shape=vertexNum)
    f=np.zeros(shape=vertexNum)

    time=0

    def _dfs_visit(v):
        nonlocal time

        mark[v]=1 # 标记为访问中
        time+=1
        d[v]=time

        # do something to v here...

        for u in range(vertexNum):
            if adjM[v, u]==1 and mark[u]==0:
                pred[u]=v
                _dfs_visit(u)

        mark[v]=2
        time+=1
        f[v]=time


def DFS_Global(adjM): # 全局DFS
    vertexNum=np.shape(adjM)[0]

    mark=np.zeros(shape=vertexNum)
    pred=np.full(shape=vertexNum, fill_value=-1)
    d=np.zeros(shape=vertexNum)
    f=np.zeros(shape=vertexNum)

    time=0

    def _dfs_visit(v):
        nonlocal time

        mark[v]=1
        time+=1
        d[v]=time

        # TODO: 在这里添加对节点v的处理逻辑
        # 示例: print(f"Processing node {v}")

        for u in range(vertexNum):
            if adjM[v, u]==1 and mark[u]==0:
                pred[u]=v
                _dfs_visit(u)

        mark[v]=2
        time+=1
        f[v]=time

    for v in range(vertexNum):
        if mark[v]==0:
            _dfs_visit(v)
'''


'\n自己写的不太好的代码\ndef DFS_from_certain_node(adjM:np.ndarray, start_id): # 从某个结点开始DFS\n    vertexNum=np.shape(adjM)[0]\n\n    mark=np.zeros(shape=vertexNum)\n    pred=np.full(shape=vertexNum, fill_value=-1)\n    d=np.zeros(shape=vertexNum)\n    f=np.zeros(shape=vertexNum)\n\n    time=0\n\n    def _dfs_visit(v):\n        nonlocal time\n\n        mark[v]=1 # 标记为访问中\n        time+=1\n        d[v]=time\n\n        # do something to v here...\n\n        for u in range(vertexNum):\n            if adjM[v, u]==1 and mark[u]==0:\n                pred[u]=v\n                _dfs_visit(u)\n\n        mark[v]=2\n        time+=1\n        f[v]=time\n\n\ndef DFS_Global(adjM): # 全局DFS\n    vertexNum=np.shape(adjM)[0]\n\n    mark=np.zeros(shape=vertexNum)\n    pred=np.full(shape=vertexNum, fill_value=-1)\n    d=np.zeros(shape=vertexNum)\n    f=np.zeros(shape=vertexNum)\n\n    time=0\n\n    def _dfs_visit(v):\n        nonlocal time\n\n        mark[v]=1\n        time+=1\n        d[v]=time\n\n        # TODO: 在这里

In [15]:
def DFS(adjM:np.ndarray, start_id:Optional[int]=None)->tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    '''
    统一的DFS遍历函数,支持指定起点或全局遍历(处理非连通图)
    
    参数:
        adjM: 邻接矩阵 (nxn), 元素为0或1
        start_id: 起始节点ID(0-based),默认None表示全局遍历(处理所有连通分量)

    返回:
        mark: 标记数组(0:未访问, 1:访问中, 2:已访问完毕), dtype=int
        pred: 前驱节点数组,各节点的前驱节点ID(无前驱为-1), dtype=int
        d: 发现时间数组(节点首次被访问的时间戳), dtype=int; 不可达/未访问节点为0
        f: 完成时间数组(节点所有邻接节点处理完毕的时间戳), dtype=int;不可达/未访问节点为0

    复杂度分析:
        只对一个结点v进行DFS, 复杂度 O(1+deg(v)), 其中 deg(v)是 v 的度
        全图 DFS, 复杂度 O(|V|+|E|), |V|, |E|分别表示图的顶点数,边数
    '''

    vertexNum=np.shape(adjM)[0]
    if vertexNum == 0:
        return np.array([]), np.array([]), np.array([]), np.array([])

    mark=np.zeros(shape=vertexNum, dtype=int)
    pred=np.full(shape=vertexNum, fill_value=-1, dtype=int)
    d=np.zeros(shape=vertexNum, dtype=int)
    f=np.zeros(shape=vertexNum, dtype=int)

    time=0

    def _dfs_visit(v):
        nonlocal time

        mark[v]=1
        time+=1
        d[v]=time

        # TODO: 在这里添加对节点v的处理逻辑
        # 示例: print(f"Processing node {v}")

        for u in range(vertexNum):
            if adjM[v, u]==1 and mark[u]==0:
                pred[u]=v
                _dfs_visit(u) # 递归访问u

        mark[v]=2
        time+=1
        f[v]=time

    if start_id is not None:
        if 0 <= start_id <vertexNum:
            _dfs_visit(start_id)
        else:
            raise ValueError(f"start_id必须在[0, {vertexNum-1}]范围内")
    else:
        for v in range(vertexNum):
            if mark[v]==0:
                _dfs_visit(v)

    print(mark, pred, d, f)
    return mark, pred, d, f
    

In [ ]:
def check_loop_DFS(adjM:np.ndarray, start_id:Optional[int]=None)->bool:
    '''
    查找有向图中是否有环. 对于无向图, 会把双向边误判为有环.所以下面的代码不可以对无向图进行搜索
    支持检查全图/只是检查start_id的连通分量中是否有环
    如果有, 返回 True, 否则返回 False.

    不能使用上边的 build_adjM 进行构建邻接矩阵, 那个是用于构建无向图的, 如果要用的话, 需要修改下面这里:
    for (u, v) in edges:
        adjM[u, v]=1    原本为:adjM[u, v]=adjM[v, u]=1 # 无向无权图
    
    参数:
        adjM: 邻接矩阵 (nxn), 元素为0或1
        start_id: 起始节点ID(0-based),默认None表示全局遍历(处理所有连通分量)

    检查原理:
        本质是在对结点进行 DFS. 如果在有向图上 对一个节点 u 进行DFS, 发现可达之前已经访问过的节点v, 那么存在一个环, 其路径为u->v->...->u.
    
    复杂度分析:
        如果是全局遍历检查是否有环, 则复杂度 O(|V|+|E|), |V|, |E|分别表示图的顶点数,边数
    '''

    vertexNum=np.shape(adjM)[0]
    if vertexNum <= 1:
        return False

    mark=np.zeros(shape=vertexNum, dtype=int)

    def _check_loop_dfs_visit(v):
    
        mark[v]=1

        for u in range(vertexNum):
            if adjM[v, u]==1:
                if mark[u]==1:
                    return True
                elif mark[u]==0:
                    if _check_loop_dfs_visit(u)==True: # 递归访问u
                        return True
                    
        mark[v]=2 # 这里很关键! 
        # 最好要设置 0, 1, 2 三个 mark 状态.
        # 两个状态很难完成正确的检查环(底层原理: 后向边), 逻辑很复杂

        return False

    if start_id is not None:
        if 0<=start_id<vertexNum:
            return _check_loop_dfs_visit(start_id)
        else:
            print(f'start_id:{start_id} out of vertexNum range.')
            return False
    else:
        for v in range(vertexNum):
            if mark[v]==0:
                if _check_loop_dfs_visit(v)==True:
                    return True
        return False
    

In [ ]:
# test check_loop
VertexNum=6
edges=[(0, 1), (1, 2), (1, 3), (2, 4), (3, 4), (4, 5)]
adjM=build_adjM(VertexNum, edges)
'''
不能使用上边的 build_adjM 进行构建邻接矩阵.
这里的代码只适用于检测有向图中是否有环.
但那个build_adjM是用于构建无向图的, 如果要用的话, 需要修改下面这里:
for (u, v) in edges:
    adjM[u, v]=1    原本为:adjM[u, v]=adjM[v, u]=1 # 无向无权图
'''
check_loop_DFS(adjM)


False

In [ ]:
def topological_sort(adjM:np.ndarray)->list:

    '''
        [问题]拓扑排序
            对有向无环图(Directed Acyclic Graph, DAG)进行拓扑排序. 
            拓扑排序的定义:
                生成一个节点序列, 满足有向无环图中任意一条有向边 e=(u, v), 在序列中 u 始终位于 v 之前。
            
            核心意义:解决「依赖优先级」问题(如任务调度, 编译依赖, 自动微分的计算顺序等)

            应用场景:
                - 任务调度:按依赖关系安排任务执行顺序(如先完成前置任务, 再执行后续任务);
                - 编译优化:编译器按代码依赖关系排序, 确保变量/函数定义在使用前;
                - 自动微分:如 Andrew Karpathy 的 MicroGrad 教程中, 按计算图的依赖顺序反向传播求导。
        [解决思路]
            Kahn 算法, 基于 BFS

            有向无环图(DAG)可抽象为「含依赖关系的任务集合」:每个节点代表一个任务, 有向边 e=(u, v) 严格表示「任务 v 的执行依赖于任务 u 的完成」—— 即必须先完成 u, 才能启动 v(u 是 v 的前置依赖任务, v 是 u 的后置依赖任务)。
            生成拓扑排序, 本质是找到一个满足所有依赖约束的任务执行序列:对于图中每一条有向边 (u, v), 序列中 u 的位置必然在 v 之前, 确保所有任务都能在其前置依赖完全满足后再执行, 最终实现无冲突、按依赖优先级推进的整体流程。

            实现思路():
            1. 环检测预处理:先通过 DFS 检测图中是否有环, 有环则无法拓扑排序, 直接返回空列表;
            2. 计算入度:入度是节点的「前置依赖数」(即指向该节点的边数, 入度=0 表示无前置依赖);
            3. 初始化队列:将所有入度=0 的节点入队(这些节点可直接执行, 无依赖约束);
            4. 迭代处理:
                a. 出队一个节点, 加入拓扑序列;
                b. 删除该节点的所有出边(即其邻接节点的入度减1, 代表解除一个前置依赖);
                c. 若邻接节点入度变为0(所有前置依赖已满足), 则入队;
            5. 终止条件:队列为空, 此时拓扑序列长度等于节点数


        [参数] adjM: 有向无环图的邻接矩阵 (nxn), 元素为0或1   
        [返回值]topo_list:一个列表, 存储拓扑排序序列.
        [复杂度分析] 
            代码首先要判断图中是否有环, 根据上面的检查环的代码, 复杂度 O(|V|+|E|).
            随后遍历所有节点, 统计其结点度(复杂度 O(|V|^2))
            在之后, 初始化队列后, 给其中添加所有入度为 0 的结点, 复杂度O(|V|)
            while 循环中嵌套 for 循环, 整体上复杂度 O(|V|+|E|)
            
            故整体复杂度: O(|V|^2 + |E|), |V|=节点数, |E|=边数 
            (如果能够通过一定方法先构建出每个结点的入度, 则复杂度不会达到 O(|V|^2), 可降至 O(|V|+|E|)

        [正确性检验]
            拓扑排序代码已通过 leetcode 210- 课程表 II 题检验:
            https://leetcode.cn/problems/course-schedule-ii/description/?envType=problem-list-v2&envId=graph

            这部分代码检验时, 需要修改上面 adjM 构建逻辑, 使之为有向图, 即只保留 adj[u][v]=1, 删掉 adj[v][u]=1
            否则会直接把无向图当成有向图, 除法 [图中存在环] 报错, 输出空列表
    '''

    if check_loop_DFS(adjM=adjM, start_id=None)==True: # 拓扑排序的前提: 图中无环
        print('the graph has loop. can\'t done topological sort.')
        return []
    
    vertexNum=adjM.shape[0]
    in_degree=[0 for _ in range(vertexNum)]

    topo_list=[]

    for v in range(vertexNum):
        tmp=0
        for u in range(vertexNum):
            if adjM[u, v]==1:
                tmp+=1
        in_degree[v]=tmp

    que=deque()

    for v in range(vertexNum):
        if in_degree[v]==0:
            que.append(v)
    
    while que:
        v=que.popleft()
        topo_list.append(v)
        for u in range(vertexNum):
            if adjM[v][u]==1:
                in_degree[u]-=1
                if in_degree[u]==0:
                    que.append(u)

    return topo_list

In [ ]:
# test_case

VertexNum=8
Edges=[(0, 4), (0, 1), (1, 5), (2, 5), (5, 6), (2, 6), (2, 3), (3, 6), (6, 7), (3, 7), (4, 3)]

adjM=build_adjM(VertexNum, Edges)
BFS(adjM=adjM, start_id=1)
DFS(adjM=adjM, start_id=0)


'''
这部分代码检验时, 需要修改上面 adjM 构建逻辑, 使之为有向图, 即只保留 adj[u][v]=1, 删掉 adj[v][u]=1
topo_list=topological_sort(adjM)
print(topo_list)
拓扑排序代码已通过 leetcode 210: 课程表 II 题检验
'''

[2 2 0 2 2 2 2 2] [-1  0 -1  4  0  1  5  6] [ 1  2  0 11 10  3  4  5] [14  9  0 12 13  8  7  6]
[0, 2, 1, 4, 5, 3, 6, 7]
